In [1]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [2]:
import onnx

# Load the model
model = onnx.load("./hair_classifier_v1.onnx")

# Get the graph
graph = model.graph

# 1. Check declared outputs (gives you the name and shape)
print("--- Model Outputs ---")
for output in graph.output:
    print(f"Name: {output.name}, Shape: {output.type.tensor_type.shape}")

# 2. Inspect the last few nodes to see the operations
print("\n--- Final Operations ---")
# We look at the last 3 nodes to see the flow
for node in graph.node[-3:]:
    print(f"Op Type: {node.op_type}, Output Name: {node.output}")

--- Model Outputs ---
Name: output, Shape: dim {
  dim_param: "s77"
}
dim {
  dim_value: 1
}


--- Final Operations ---
Op Type: Gemm, Output Name: ['linear']
Op Type: Relu, Output Name: ['relu_1']
Op Type: Gemm, Output Name: ['output']


In [3]:
import numpy as np

# Download and prepare the image
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'

# Download the image
img = download_image(url)

# Prepare the image with target size (299, 299) - typical for image classification models
target_size = (200, 200)
img_prepared = prepare_image(img, target_size)

# Convert to numpy array
img_array = np.array(img_prepared)

# Check the first pixel's R channel value
first_pixel_r_channel = img_array[0, 0, 0]

print(f"Image shape after preparation: {img_array.shape}")
print(f"First pixel RGB values: R={img_array[0, 0, 0]}, G={img_array[0, 0, 1]}, B={img_array[0, 0, 2]}")
print(f"\n🎯 ANSWER: First pixel R channel value = {first_pixel_r_channel}")

# Additional information
print(f"\nData type: {img_array.dtype}")
print(f"Value range: [{img_array.min()}, {img_array.max()}]")

# If the model expects normalized values (0-1 range)
img_array_normalized = img_array / 255.0

first_pixel_r_normalized = img_array_normalized[0, 0, 0]

print(f"\nWith normalization:")
print(f"First pixel R channel (normalized) = {first_pixel_r_normalized:.6f}")

# Complete analysis of the first pixel
print("="*50)
print("FIRST PIXEL ANALYSIS")
print("="*50)

# Original values (0-255)
print(f"Original RGB: [{img_array[0, 0, 0]}, {img_array[0, 0, 1]}, {img_array[0, 0, 2]}]")

# Normalized values (0-1)
print(f"Normalized RGB: [{img_array_normalized[0, 0, 0]:.6f}, {img_array_normalized[0, 0, 1]:.6f}, {img_array_normalized[0, 0, 2]:.6f}]")

# Answer
print(f"\n🎯 First pixel R channel = {img_array[0, 0, 0]}")

Image shape after preparation: (200, 200, 3)
First pixel RGB values: R=61, G=104, B=22

🎯 ANSWER: First pixel R channel value = 61

Data type: uint8
Value range: [0, 255]

With normalization:
First pixel R channel (normalized) = 0.239216
FIRST PIXEL ANALYSIS
Original RGB: [61, 104, 22]
Normalized RGB: [0.239216, 0.407843, 0.086275]

🎯 First pixel R channel = 61


In [5]:
import onnxruntime as ort
import numpy as np

# Step 1: Load the ONNX model
model_path = "./hair_classifier_v1.onnx"
session = ort.InferenceSession(model_path)

# Step 2: Get model input/output information
print("="*60)
print("MODEL INFORMATION")
print("="*60)

# Input information
input_info = session.get_inputs()[0]
print(f"Input name: {input_info.name}")
print(f"Input shape: {input_info.shape}")
print(f"Input type: {input_info.type}")

# Output information
output_info = session.get_outputs()[0]
print(f"\nOutput name: {output_info.name}")
print(f"Output shape: {output_info.shape}")
print(f"Output type: {output_info.type}")

# Step 3: Prepare your image
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'
img = download_image(url)

# Use the target size from model input shape (usually [batch, channels, height, width])
# For example, if input shape is [1, 3, 299, 299]
target_size = (200, 200)  # Adjust based on your model
img_prepared = prepare_image(img, target_size)

# Step 4: Convert to numpy array and preprocess
img_array = np.array(img_prepared, dtype=np.float32)

# Normalize to [0, 1] range
img_array = img_array / 255.0

# Transpose from (height, width, channels) to (channels, height, width)
# This is required for most deep learning models
img_array = img_array.transpose(2, 0, 1)

# Add batch dimension: (channels, height, width) -> (batch, channels, height, width)
img_array = np.expand_dims(img_array, axis=0)

print(f"\n{'='*60}")
print("PREPARED INPUT")
print("="*60)
print(f"Final input shape: {img_array.shape}")
print(f"Input dtype: {img_array.dtype}")
print(f"Input range: [{img_array.min():.4f}, {img_array.max():.4f}]")

# Step 5: Run inference
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

outputs = session.run([output_name], {input_name: img_array})

# Step 6: Get and process the output
prediction = outputs[0]

print(f"\n{'='*60}")
print("MODEL OUTPUT")
print("="*60)
print(f"Raw output shape: {prediction.shape}")
print(f"Raw output: {prediction}")

# For classification, apply softmax to get probabilities
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum()

if prediction.ndim > 1:
    probabilities = softmax(prediction[0])
else:
    probabilities = softmax(prediction)

predicted_class = np.argmax(probabilities)

print(f"\nProbabilities: {probabilities}")
print(f"Predicted class: {predicted_class}")
print(f"Confidence: {probabilities[predicted_class]:.4f}")

# If it's binary classification (2 classes)
if len(probabilities) == 2:
    print(f"\nClass 0 probability: {probabilities[0]:.4f}")
    print(f"Class 1 probability: {probabilities[1]:.4f}")

MODEL INFORMATION
Input name: input
Input shape: ['s77', 3, 200, 200]
Input type: tensor(float)

Output name: output
Output shape: ['s77', 1]
Output type: tensor(float)

PREPARED INPUT
Final input shape: (1, 3, 200, 200)
Input dtype: float32
Input range: [0.0000, 1.0000]

MODEL OUTPUT
Raw output shape: (1, 1)
Raw output: [[1.9222327]]

Probabilities: [1.]
Predicted class: 0
Confidence: 1.0000


In [9]:
def predict_with_onnx(model_path, image_url, target_size=(299, 299)):
    """
    Complete pipeline for ONNX model inference
    
    Parameters:
    model_path (str): Path to .onnx model
    image_url (str): URL of the image
    target_size (tuple): Target size for image (height, width)
    
    Returns:
    dict: Prediction results
    """
    
    # Load model
    session = ort.InferenceSession(model_path)
    
    # Download and prepare image
    img = download_image(image_url)
    img_prepared = prepare_image(img, target_size)
    
    # Convert to array and preprocess
    img_array = np.array(img_prepared, dtype=np.float32)
    img_array = img_array / 255.0  # Normalize
    img_array = img_array.transpose(2, 0, 1)  # HWC -> CHW
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Run inference
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    outputs = session.run([output_name], {input_name: img_array})
    
    # Process output
    prediction = outputs[0]
    
    # Apply softmax
    def softmax(x):
        exp_x = np.exp(x - np.max(x))
        return exp_x / exp_x.sum()
    
    if prediction.ndim > 1:
        probabilities = softmax(prediction[0])
    else:
        probabilities = softmax(prediction)
    
    predicted_class = np.argmax(probabilities)
    
    return {
        "predicted_class": int(predicted_class),
        "probabilities": probabilities.tolist(),
        "confidence": float(probabilities[predicted_class]),
        "raw_output": prediction.tolist()
    }

# Usage
result = predict_with_onnx(
    model_path="./hair_classifier_v1.onnx",
    image_url='https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg',
    target_size=(200, 200)
)

print("🎯 PREDICTION RESULT:")
print(f"Predicted class: {result['predicted_class']}")
print(f"Confidence: {result['confidence']:.4f}")
print(f"All probabilities: {result['probabilities']}")

🎯 PREDICTION RESULT:
Predicted class: 0
Confidence: 1.0000
All probabilities: [1.0]
